In [ ]:
''' FOR DEVELOPMENT ONLY, DELETE CELL '''

# set auto reload imported modules tagged
%load_ext autoreload
%autoreload 2

# data array processing
import numpy as np
#import pandas as pd

# plotting with matplotlib
#%matplotlib widget
from matplotlib import pyplot as plt

## development override params
pvlibs_path = '/home/brendan/dev/pvlibs'
base_path = '../data/slt'


In [ ]:
''' initialise notebook '''

# add location to path
#pvlibs_path = '/srv/pvlibs'
import sys
sys.path.append(pvlibs_path)

# plotting with matplotlib, set interactive figure display
%matplotlib widget

# import pvlibs package
import pvlibs


In [ ]:
''' import measurement files '''

# define directory to search; "./" for current directory, "../" for up one directory
#base_path = './'

# define measurement properties (!required!)
props = {
    # measurement type
    'meas_type': 'slt', # 'slt' for sinton lifetime
    'file_ext': 'ltr', # ['ltr'|'xlsm']
    'file_type': 'ltr', # ['ltr'|'xlsm']
    
    # wafer properties
    'wafer_doping_type': 'p-type', # ['p-type'|'n-type']
    'wafer_resistivity': 2.0, # [Ohm.cm^2]
    'wafer_thickness': .018, # [cm]
    'wafer_optical_const': 1.0, # []
    
    # measurement conditions
    'temperature': 298., # [K]
    'illumination_mode': 'gen', # ['gen'|'trans']
}

# generate measurement file database
db = pvlibs.init_file_db(base_path, props)


In [ ]:
''' parse parameters from file name '''

# example: "h555_fired-830C_D23_TC.ltr"

# define single parameter separator; e.g. '-', '_', or '\s' (space)
param_sep = '-'

# define ordered list of parameters within filename separated by param_sep
params = [
    'misc1',
    'misc2',
    'proc_temp',
    'state',
    'proc_time'
    'group',
    'device',
    'region',
]

# parse file name parameters from each measurement file
db = pvlibs.parse_file_names(db, param_sep, params)


In [ ]:
''' import measurement data '''

# import data from files
db = pvlibs.import_file_data(db)


In [ ]:
''' process measurement data '''

# process measurement data
db = pvlibs.process_file_data(db)


In [ ]:
''' model sinton lifetime data '''

# set measurement process type; 'mlt' for model sinton lifetime
meas_type = 'mlt'

# define parameters required for process;
# 'nd_range' as tuple(min, max) for model fiting charge density range
# 'nd_vals' as charge density for parameter extraction (tau_eff, tau_blk), exclude for default (10% doping)
params = {
    'nd_range': [ 1.0e14, 1.0e17 ],
    'nd_vals': 2.0e15,
}

# process measurement data
db = pvlibs.process_file_data(db, meas_type, params)


In [ ]:
''' plot sinton lifetime model fit '''

# define parameter values to filter by for single node
params = {
    'device': '09',
    'region': '01',
}

# default plot sinton lifetime model fit; inc. components and print of fit params
pvlibs.plot_mlt_fit(db, params)


In [ ]:
''' save model fit plots '''

# define ordered list of parameters to use for output plot file name
params = [
    'device',
    'region',
]

# define output file name header
file_name_head = './results/lt-fit'

# prepare and save each measurement plot to disk
pvlibs.save_mlt_fit(db, file_name_head, params)


In [ ]:
''' compile and export data'''

# define list of measurement labels to export; format each as 'file_name_param': 'Output Label'
labels = {
    'device': 'Device',
    'region': 'Region',
}

# select values to export; format each as 'param': 'Output Label'
values = {
    'ivoc': 'iVoc [V]',
    'J_0': 'J_0 [A]',
    't_eff': 'Tau_Eff [s]',
    't_blk': 'Tau_Blk [s]',
    'R2': 'Fit [R^2]',
    'k_val': 'SRH k-value []',
    'pFF': 'pseudo-FF []',
}

# define results output file name
file_name = './results/lt-fit-summary'

# compile dataset by selected labels and values
data = pvlibs.compile_data(db, labels, values, file_name)


In [ ]:
''' save all data '''

# define ordered list of parameters to use for data output file name
params = [
    'device',
    'region',
]

# define output file name header
file_name_head = './results/lt-fit'

# prepare and save all measurement and calculated data to file
pvlibs.save_all_data(db, file_name_head, params)
